<a href="https://colab.research.google.com/github/zaghambajwa11/Highlight-Duplicate-Values-In-SpreadSheet/blob/main/highlightDuplicateFromColumn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import openpyxl
import random

def find_duplicates(xlsx_file, column_name):
    """
    Finds duplicate values in an Excel file for a specific column and returns their addresses in the form of A1, B2, etc.
    """
    df = pd.read_excel(xlsx_file)
    duplicates = df.duplicated(subset=[column_name], keep=False)
    df[column_name] = df[column_name].astype(str)  # convert column to string
    colors = {}
    style = df.style.apply(lambda x: [f'background-color: {colors[x.iloc[i]]}' if x.iloc[i] in colors else '' for i in range(len(x))], axis=1, subset=[column_name])
    for index, value in df[column_name].duplicated(keep=False).items():
        if value:
            cell_value = df.loc[index, column_name]
            if cell_value not in colors:
                colors[cell_value] = f'#{random.randint(0, 0xffffff):06x}'
    writer = pd.ExcelWriter('output.xlsx', engine='openpyxl')
    writer.book = openpyxl.load_workbook(xlsx_file)
    style.to_excel(writer, sheet_name='Duplicates', index=False)
    writer.save()
    addresses = []
    for index, row in duplicates.items():
        if row:
            cell_address = openpyxl.utils.get_column_letter(df.columns.get_loc(column_name)+1) + str(index+1)
            prev_address = openpyxl.utils.get_column_letter(df.columns.get_loc(column_name)+1) + str(df[df[column_name]==df.iloc[index][column_name]].index.tolist()[0]+1)
            addresses.append((cell_address, prev_address))
    return addresses

# example usage
xlsx_file = 'data.xlsx'
column_name = 'Driver'
addresses = find_duplicates(xlsx_file, column_name)
if len(addresses) > 0:
    print(f'Duplicate values found in column "{column_name}" at addresses:')
    for address in addresses:
        print(f'{address[0]} and {address[1]}')
else:
    print(f'No duplicate values found in column "{column_name}".')
